# Homework 1

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data

In [1]:
import pandas as pd

In [2]:
!wget -P ~/data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!wget -P ~/data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-16 16:52:18--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.189.87, 13.225.189.130, 13.225.189.175, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.189.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘/home/ubuntu/data/yellow_tripdata_2022-01.parquet.1’

yellow_tripdata_202 100%[===================>]  36.37M  47.2MB/s    in 0.8s    

2023-05-16 16:52:19 (47.2 MB/s) - ‘/home/ubuntu/data/yellow_tripdata_2022-01.parquet.1’ saved [38139949/38139949]

--2023-05-16 16:52:19--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.189.87, 13.225.189.178, 13.225.189.175, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.c

In [3]:
df_jan = pd.read_parquet('~/data/yellow_tripdata_2022-01.parquet')
df_feb = pd.read_parquet('~/data/yellow_tripdata_2022-02.parquet')

In [4]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
nrows = df_jan.shape[0]
nrows

2463931

In [53]:
with pd.option_context("display.max_columns", None):
    display(df_jan.head())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## Q2. Computing duration

In [6]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_jan['duration'].std()

46.44530513776499

## Q3. Dropping outliers

In [8]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_jan.shape[0]/nrows

0.9827547930522406

## Q4. One-hot encoding

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
categorical = ['PULocationID', 'DOLocationID']
dict_features = df_jan[categorical].astype(str).to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(dict_features)

In [11]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [12]:
print(f"Number of explicitly-stored values={X_train.getnnz()}")
print(f"Number of rows={X_train.shape[0]}")
print(f"Number of columns={X_train.shape[1]}")

Number of explicitly-stored values=4842880
Number of rows=2421440
Number of columns=515


## Q5. Training a model

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [14]:
target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190135965426

## Q6. Evaluating the model

In [16]:
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds() / 60

In [17]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [18]:
valid_dicts = df_feb[categorical].astype("str").to_dict(orient="records")
X_valid = dv.transform(valid_dicts)

In [19]:
X_valid

<2918187x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5836368 stored elements in Compressed Sparse Row format>

In [20]:
y_valid = df_feb["duration"]

In [21]:
y_pred = lr.predict(X_valid)
mean_squared_error(y_valid, y_pred, squared=False)

7.786389467483621